In [1]:
print("hello world")

hello world


In [2]:
import os

In [3]:
%pwd

'/workspaces/Text-Summarizer-using-Hugging-face-and-Transformers/research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'/workspaces/Text-Summarizer-using-Hugging-face-and-Transformers'

In [6]:
# create entity with dataclasses where the values are held
from dataclasses import dataclass
from pathlib import Path 

# input and output for DataIngestioncomponent
@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: Path
    local_data_file: Path
    unzip_dir: Path

In [7]:
# since we got the input and output director, we need to read the yaml and store them as value
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,      # loading from constants
                 params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        # creating artifacts folder/ parent output folder
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)->DataIngestionConfig:
        config = self.config.data_ingestion  # reading all the necessary file within the config.yaml/data_ingestion
        create_directories([config.root_dir])  # creating the parent directory for data_ingestion component

        data_ingestion_config = DataIngestionConfig(
            root_dir= config.root_dir,
            source_URL=  config.source_URL,
            local_data_file= config.local_data_file,
            unzip_dir= config.unzip_dir,
        )
        return data_ingestion_config

In [8]:
# creating the component
import os
import urllib.request as request
import zipfile 

from src.textSummarizer.logging import logger
from src.textSummarizer.utils.common import *

class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config = config

    def download_file(self):
        """
        input/arguments: DataIngestionConfig class instance
        process: Downloads all the resources mentioned in the url --> unzip --> store in local repository
        output: Downloaded data in the local repo
        """

        if not os.path.exists(self.config.local_data_file):    # create local data file if it does not exist
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"File downloaded and stored in local Successfully!!!")
        else:
            logger.info(f"File already Exists!! ")

    
    def extract_zip_file(self):
        """
        This extracts the zip_file_path into the data directory
        and this returns None
        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok = True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [9]:
config = ConfigurationManager()
data_ingestion_config= config.get_data_ingestion_config()
data_ingestion_comp = DataIngestion(data_ingestion_config)

data_ingestion_comp.download_file()
data_ingestion_comp.extract_zip_file()

[2026-01-20 05:24:53 | INFO | summarizerLogger | common.py:60 | Created directory at: artifacts]
[2026-01-20 05:24:53 | INFO | summarizerLogger | common.py:60 | Created directory at: artifacts/data_ingestion]


[2026-01-20 05:24:55 | INFO | summarizerLogger | 1669548521.py:25 | File downloaded and stored in local Successfully!!!]
